# VoF

See "Volume of Fluid Method: A Brief Review" by A. Mohan and G. Tomar.

In [ ]:
import numpy as np

In [ ]:
ndim = 2
grid_shape = (10, 10)

In [ ]:
eps = 1e-4

**Axes and orientation**: For simplicity, the rows of matrices correspond to the x-axis and the columns correspond to the y-axis.
For ease of plotting, the x-axis is considered to be pointing "right" and the y-axis to be pointing "upward" in the physics (right vs. left doesn't matter for the physics, so this just means that gravity points to negative y's).
This means that `imshow` output is wrong unless the axes have been explicitly permuted. Writing a matrix out, like `print(x)`, also orients the axes wrong.

In [ ]:
from typing import Literal

x_grid, y_grid = np.arange(grid_shape[0]), np.arange(grid_shape[1])
x_stagger = np.arange(-1/2, grid_shape[0] + 1/2)
y_stagger = np.arange(-1/2, grid_shape[1] + 1/2)
y_vgrid, x_vgrid = np.meshgrid(x_stagger, y_grid)
y_ugrid, x_ugrid = np.meshgrid(x_grid, y_stagger)
y_grid, x_grid = np.meshgrid(x_grid, y_grid)

def reset_field(shape: Literal['dambreak', 'circle', 'checkerboard', 'surface']):
    field = {
        'f': np.zeros(grid_shape),
        'u': np.zeros((grid_shape[0] + 1, grid_shape[1])),
        'v': np.zeros((grid_shape[0], grid_shape[1] + 1)),
    }
    center = (grid_shape[0]/2, grid_shape[1]/2)
    if shape == 'circle':
        field['f'][(x_grid-center[0])**2 + (y_grid-center[1])**2 < 10] = 1
    elif shape == 'dambreak':
        field['f'][x_grid < grid_shape[0]/2] = 1
        field['f'][x_grid == grid_shape[0]/2] = 0.5
    elif shape == 'surface':
        field['f'][y_grid < grid_shape[0]/2] = 1
        field['f'][y_grid == grid_shape[0]/2] = 0.5
    elif shape == 'checkerboard':
        field['f'][(x_grid & 1) ^ (y_grid & 1) == 0] = 1
    return field
field = reset_field('surface')

In [ ]:
class Direction:
    def __init__(self, idx):
        self.idx = idx

    @property
    def earlier(self):
        earlier = [slice(None), slice(None)]
        earlier[self.idx] = slice(None, -1)
        return tuple(earlier)

    @property
    def later(self):
        earlier = [slice(None), slice(None)]
        earlier[self.idx] = slice(1, None)
        return tuple(earlier)

    @property
    def inner(self):
        earlier = [slice(None), slice(None)]
        earlier[self.idx] = slice(1, -1)
        return tuple(earlier)

class d_:
    x = Direction(0)
    y = Direction(1)

def deriv(field, d: Direction, dx: float):
    return (field[d.later] - field[d.earlier]) / dx

def avg(field, d):
    return (field[d.later] + field[d.earlier]) / 2

def expand0(field, d):
    shape = list(field.shape)
    shape[d.idx] += 2
    result = np.zeros(shape)
    result[d.inner] = field
    return result

def avgpad(field, d):
    shape = list(field.shape)
    shape[d.idx] += 1
    result = np.zeros(shape)
    result[d.earlier] = field
    result[d.later] = field
    result[d.inner] = avg(field, d)
    return result

In [ ]:
import matplotlib.pyplot as plt

def plot_img(field: np.ndarray, fig_ax=None):
    if fig_ax is None:
        fig, ax = plt.subplots()
    else:
        fig, ax = fig_ax
    m = ax.matshow(field.T)
    ax.invert_yaxis()
    fig.colorbar(m)
    return m

def plot_field(field, fig_and_axes=None):
    if fig_and_axes is not None:
        fig, axes = fig_and_axes
    else:
        fig, axes = plt.subplots(1, 2)
    board = plot_img(field['f'], (fig, axes[0]))
    u = avg(field['u'], d_.x)
    v = avg(field['v'], d_.y)
    quiver = axes[1].quiver(x_grid, y_grid, u, v)
    return fig, board, quiver

plot_field(field)

## Solving the Navier-Stokes equation via an explicit projection method
See <https://web.stanford.edu/class/me469b/handouts/incompressible.pdf>
and <https://orbi.uliege.be/bitstream/2268/2649/1/BBEC9106.pdf> for the projection method.

### Theory

Incompressible Euler equations, i.e. incompressible, adiabatic, inviscid flow (from [Wikipedia](https://en.wikipedia.org/wiki/Euler_equations_(fluid_dynamics)#Incompressible_Euler_equations)):

\begin{align}
\frac{\partial \rho}{\partial t} + \mathbf{u} \cdot \nabla\rho &= 0 \\
\frac{\partial \mathbf{u}}{\partial t} + \mathbf{u} \cdot \nabla \mathbf{u} &= - \frac{\nabla p}{\rho} + \mathbf{f} \\
\nabla \cdot \mathbf{u} &= 0
\end{align}

In our case, we have a no-slip boundary condition at the walls: $$\vec{n} \cdot \mathbf{u} |_{\partial \Omega} = 0$$

The first equation (transport of density) is handled by VoF, because VoF is what we use to model density. But first, we need to compute the velocity (with all other equations and boundary conditions). To solve (ii) and (iii) at the same time, we use a projection method: we first compute a transport velocity $u_\text{trans}$ that solves the first equation without the pressure term, then we compute what the pressure must be to fulfill both (ii) and (iii). In equations:

We partly discretize the equations with an explicit scheme into

\begin{align}
\frac{\mathbf{u}^{(i+1)} - \mathbf{u}^{(i)}}{\Delta t} + \mathbf{u}^{(i)} \cdot \nabla \mathbf{u}^{(i)} &= - \frac{\nabla p}{\rho} + \mathbf{f} \\
\nabla \cdot \mathbf{u}^{(i+1)} &= 0
\end{align}

\begin{align}
\mathbf{u}^{(i+1)} &= \mathbf{u}^{(i)} + \Delta t ( \mathbf{u}^{(i)} \cdot \nabla \mathbf{u}^{(i)} ) - \Delta t \frac{\nabla p}{\rho} + \Delta t \mathbf{f} \\
\nabla \cdot \mathbf{u}^{(i+1)} &= 0
\end{align}

Define $\mathbf{u}_\text{trans} = \mathbf{u}^{(i)} + \Delta t ( \mathbf{u}^{(i)} \cdot \nabla \mathbf{u}^{(i)} ) + \Delta t \mathbf{f}$, i.e. a "transport" velocity field which can be computed explicitly, but is not divergence-free. Then

\begin{align}
\mathbf{u}^{(i+1)} &= \mathbf{u}_\text{trans} - \Delta t \frac{\nabla p}{\rho} \\
\nabla \cdot \mathbf{u}^{(i+1)} &= 0 \\
\implies \nabla \cdot (\mathbf{u}_\text{trans} - \Delta t \frac{\nabla p}{\rho}) &= 0 \\
\implies \Delta t \nabla \cdot \frac{\nabla p}{\rho} &= \nabla \cdot \mathbf{u}_\text{trans}
\end{align}
where we can compute the right-hand side with finite differences. Then we can solve this differential equation and get $p$. If the density was constant, this would be a Poisson equation (but the density is not constant, so we need a slightly different stencil).

Note that $\frac{\partial u_i u_j}{\partial x_i} = \frac{\partial u_i}{\partial x_i} u_j + u_i \frac{\partial u_j}{\partial x_i} = 0 + u_i \frac{\partial u_j}{\partial x_i} = u \cdot \nabla u$.

### Transport step

In [ ]:
dx = 1 / grid_shape[0]
dt = dx / 10

In [ ]:
def rho(f):
    return 0.01 + 1 * f

def get_transport(field, rho, dt, dx):
    ui_uj = avg(field['v'], d_.y) * avg(field['u'], d_.x)
    H_i = -deriv(ui_uj, d_.y, dx)
    H_j = -deriv(ui_uj, d_.x, dx)
    H = deriv(H_i, d_.x, dx) + deriv(H_j, d_.y, dx)
    u_transport = np.zeros_like(field['u'])
    u_transport[d_.x.inner] = (
        field['u'][1:-1, :]
        + dt * avg(expand0(avg(H_i, d_.y), d_.y), d_.x)
    )
    v_transport = np.zeros_like(field['v'])
    v_transport[d_.y.inner] = (
        field['v'][:, 1:-1]
        + dt * avg(expand0(avg(H_j, d_.x), d_.x), d_.y)
        - dt * 9.81 * avg(rho, d_.y) * (dx**2)
    )
    return u_transport, v_transport

rho_field = rho(field['f'])

u_transport, v_transport = get_transport(field, rho_field, dt=dt, dx=dx)
div_transport = deriv(u_transport, d_.x, dx) + deriv(v_transport, d_.y, dx)
plt.quiver(x_ugrid, y_ugrid, u_transport, np.zeros_like(u_transport))
plt.quiver(x_vgrid, y_vgrid, np.zeros_like(v_transport), v_transport)

### Solving the pressure equation

In [ ]:
import scipy as sp

def get_poisson_matrix_with_neumann_zero_boundary_conditions(field, rho):
    rho_dx, rho_dy = avg(1 / rho, d_.x), avg(1 / rho, d_.y)
    A = sp.sparse.dok_matrix((field.size, field.size))
    rhs = field.flatten()

    def index(i, j):
        assert 0 <= i <= field.shape[0]
        assert 0 <= j <= field.shape[1]
        return i * field.shape[1] + j

    for i in range(field.shape[0]):
        for j in range(field.shape[1]):
            total = 0.0
            if j != field.shape[1]-1:
                A[index(i, j), index(i, j+1)] = rho_dy[i, j]
                total += rho_dy[i, j]
            if j != 0:
                A[index(i, j), index(i, j-1)] = rho_dy[i, j-1]
                total += rho_dy[i, j-1]
            if i != field.shape[0]-1:
                A[index(i, j), index(i+1, j)] = rho_dx[i, j]
                total += rho_dx[i, j]
            if i != 0:
                A[index(i, j), index(i-1, j)] = rho_dx[i-1, j]
                total += rho_dx[i-1, j]
            A[index(i, j), index(i, j)] = -total
            if i == 0 and j == 0:
                # Set the pressure (arbitrarily) to 0 at some point.
                # We overwrite a constraint but it should be redundant anyway.
                A[index(i, j), index(i, j)] = 1
                rhs[index(i, j)] = 0

    A = sp.sparse.dia_matrix(A)
    return A, rhs

A, rhs = get_poisson_matrix_with_neumann_zero_boundary_conditions(div_transport, rho_field)

In [ ]:
A

In [ ]:
def get_pressure(field, div_transport, rho_field, dx, dt):
    A, rhs = get_poisson_matrix_with_neumann_zero_boundary_conditions(div_transport, rho_field)
    p = sp.sparse.linalg.spsolve(A, rhs)
    p = 1 / dt * (dx**2) * p
    p = p.reshape(grid_shape)
    return p

p = get_pressure(field, div_transport, rho_field, dx=dx, dt=dt)

In [ ]:
plot_img(p)

In [ ]:
def update_velocity(field, dx, dt):
    rho_field = rho(field['f'])
    u_transport, v_transport = get_transport(field, rho_field, dx=dx, dt=dt)
    div_transport = deriv(u_transport, d_.x, dx) + deriv(v_transport, d_.y, dx)

    p = get_pressure(field, div_transport, rho_field, dx=dx, dt=dt)
    dx_p = deriv(p, d_.x, dx)
    dy_p = deriv(p, d_.y, dx)
    field['u'] = u_transport - dt * expand0(dx_p / avg(rho_field, d_.x), d_.x)
    field['v'] = v_transport - dt * expand0(dy_p / avg(rho_field, d_.y), d_.y)

In [ ]:
dx_p = expand0(deriv(p, d_.x, dx), d_.x)
dy_p = expand0(deriv(p, d_.y, dx), d_.y)
plt.quiver(x_ugrid, y_ugrid, dx_p, np.zeros_like(dx_p), alpha=0.3)
plt.quiver(x_vgrid, y_vgrid, np.zeros_like(dy_p), dy_p, alpha=0.3)

In [ ]:
update_velocity(field, dx=dx, dt=dt)
plot_field(field)

#### Surface reconstruction

In [ ]:
def check(v):
    assert not np.any(np.isnan(v))
    assert not np.any(np.isinf(v))

In [ ]:
def get_interface_normal(field):
    df_dx = -deriv(field['f'], d_.x, dx)
    df_dy = -deriv(field['f'], d_.y, dx)
    interface_normal = np.zeros((*grid_shape, ndim))
    interface_normal[:, :, 0] = avgpad(df_dx, d_.x)
    interface_normal[:, :, 1] = avgpad(df_dy, d_.y)
    return interface_normal
interface_normal = get_interface_normal(field)

In [ ]:
plt.quiver(x_grid, y_grid, interface_normal[:, :, 0], interface_normal[:, :, 1])

In [ ]:
def get_wall_sizes(field, interface_normal):
    wall_y_early = np.nan * np.ones_like(field['f'])
    wall_y_late = np.nan * np.ones_like(field['f'])
    wall_x_early = np.nan * np.ones_like(field['f'])
    wall_x_late = np.nan * np.ones_like(field['f'])

    dside_x = interface_normal[:, :, 0] / interface_normal[:, :, 1]
    dside_y = interface_normal[:, :, 1] / interface_normal[:, :, 0]
    b = np.abs(interface_normal[:, :, 0])
    a = np.abs(interface_normal[:, :, 1])
    far_side_x = field['f'] - (1 - field['f']) * (1 + b / a)
    far_side_y = field['f'] - (1 - field['f']) * (1 + a / b)
    near_side_x = field['f'] * (1 + b / a)
    near_side_y = field['f'] * (1 + a / b)
    full = (field['f'] == 1)
    empty = (field['f'] == 0)

    mask = (interface_normal[:, :, 0] > 0)
    wall_y_late[mask] = far_side_y[mask]
    wall_y_early[mask] = near_side_y[mask]
    mask = (interface_normal[:, :, 0] < 0)
    wall_y_late[mask] = near_side_y[mask]
    wall_y_early[mask] = far_side_y[mask]

    mask = (interface_normal[:, :, 1] > 0)
    wall_x_late[mask] = far_side_x[mask]
    wall_x_early[mask] = near_side_x[mask]
    mask = (interface_normal[:, :, 1] < 0)
    wall_x_late[mask] = near_side_x[mask]
    wall_x_early[mask] = far_side_x[mask]
    
    mask = (interface_normal[:, :, 1] == 0)
    wall_x_late[mask] = 0
    wall_x_early[mask] = 1
    # Only the "early" formula works in this case
    wall_y_late[mask] = wall_y_early[mask]
    dside_x[mask] = 0
    
    mask = (interface_normal[:, :, 0] == 0)
    wall_y_late[mask] = 0
    wall_y_early[mask] = 1
    wall_x_late[mask] = wall_x_early[mask]
    dside_y[mask] = 0

    mask = (field['f'] >= 1 - eps)
    wall_y_late[mask] = 1
    wall_y_early[mask] = 1
    wall_x_late[mask] = 1
    wall_y_early[mask] = 1
    
    wall_y_early = np.clip(wall_y_early, 0, 1)
    wall_y_late = np.clip(wall_y_late, 0, 1)
    wall_x_late = np.clip(wall_x_late, 0, 1)
    wall_x_early = np.clip(wall_x_early, 0, 1)

    for side in (wall_y_early, wall_y_late, wall_x_early, wall_x_late):
        side[full] = 1
        side[empty] = 0
        check(side)
    for dside in (dside_x, dside_y):
        dside[full] = 0
        dside[empty] = 0
    return wall_y_early, wall_y_late, wall_x_early, wall_x_late, dside_x, dside_y

def get_advected_volume(field, interface_normal):
    wall_y_early, wall_y_late, wall_x_early, wall_x_late, dside_x, dside_y = get_wall_sizes(field, interface_normal)
    V_y_early = field['v'][d_.y.earlier] * np.clip(wall_y_early + 0.5 * dside_y, 0, 1)
    V_y_late = -field['v'][d_.y.later] * np.clip(wall_y_late + 0.5 * dside_y, 0, 1)
    V_x_early = field['u'][d_.x.earlier] * np.clip(wall_x_early + 0.5 * dside_x, 0, 1)
    V_x_late = -field['u'][d_.x.later] * np.clip(wall_x_late + 0.5 * dside_x, 0, 1)

    return V_y_early, V_y_late, V_x_early, V_x_late

V_y_early, V_y_late, V_x_early, V_x_late = get_advected_volume(field, interface_normal)

for v in (V_y_early, V_y_late, V_x_late, V_x_late):
    check(v)

In [ ]:
def get_upwind_advected_fluxes(
    field,
    V_y_early, V_y_late, V_x_early, V_x_late
):
    V_u = np.nan * np.ones_like(field['u'])
    np.putmask(V_u[d_.x.earlier], field['u'][d_.x.earlier] < 0, V_x_early)
    np.putmask(V_u[d_.x.later], field['u'][d_.x.later] > 0, -V_x_late)
    V_u[field['u'] == 0] = 0

    V_v = np.nan * np.ones_like(field['v'])
    np.putmask(V_v[d_.y.earlier], field['v'][d_.y.earlier] < 0, V_y_early)
    np.putmask(V_v[d_.y.later], field['v'][d_.y.later] > 0, -V_y_late)
    V_v[field['v'] == 0] = 0
    return V_u, V_v

V_u, V_v = get_upwind_advected_fluxes(field, V_y_early, V_y_late, V_x_early, V_x_late)
for v in (V_v, V_u):
    check(v)

In [ ]:
def split_scheme(field, V_u, V_v):
    C_star = field['f'].copy()
    C_star -= (dt / dx) * deriv(V_u, d_.x, dx)
    C_star[field['f'] >= 0.5] += (dt / dx) * deriv(field['u'], d_.x, dx)[field['f'] >= 0.5]

    C_next = C_star.copy()
    C_next -= (dt / dx) * deriv(V_v, d_.y, dx)
    C_next[field['f'] >= 0.5] += (dt / dx) * deriv(field['v'], d_.y, dx)[field['f'] >= 0.5]
    return C_next

C_next = split_scheme(field, V_u, V_v)
check(C_next)
eps = 0.1 * dx
assert np.max(C_next) <= 1 + eps
assert np.min(C_next) >= 0 - eps
# TODO: this shouldn't be necessary
C_next = np.clip(C_next, 0, 1)

In [ ]:
np.max(C_next)

In [ ]:
plot_field({'f': C_next, 'u': field['u'], 'v': field['v']})

## Debugging

In [ ]:
field = reset_field('surface')
#navier_stokes_vof_step(field)

#update_velocity(field, dx, dt)

In [ ]:
np.all(np.abs(field['u']) <= 1e-10)

In [ ]:
np.max(np.abs(field['v']))

In [ ]:
interface_normal = get_interface_normal(field)
ax = plt.axes()
ax.quiver(x_grid, y_grid, interface_normal[:, :, 0], interface_normal[:, :, 1])
ax.imshow(field['f'].T, origin="lower")

In [ ]:
interface_normal[:, :, 1] / interface_normal[:, :, 0]

In [ ]:
def plot_quantities(y_early, y_late, x_early, x_late, offset, **kwargs):
    plt.quiver(
        x_grid - 0.5 + offset, y_grid - 0.5,
        np.zeros_like(x_early), x_early,
        headwidth=0, scale_units='x', color='red', **kwargs
    )
    plt.quiver(
        x_grid - 0.5, y_grid - 0.5 + offset,
        y_early, np.zeros_like(y_early),
        headwidth=0, scale_units='y', color='orange', **kwargs
    )
    plt.quiver(
        x_grid + 0.5 - offset, y_grid - 0.5,
        np.zeros_like(x_late), x_late,
        headwidth=0, scale_units='x', color='blue', **kwargs
    )
    plt.quiver(
        x_grid - 0.5, y_grid + 0.5 - offset,
        y_late, np.zeros_like(y_late),
        headwidth=0, scale_units='y', color='green', **kwargs
    )

wall_y_early, wall_y_late, wall_x_early, wall_x_late, dside_x, dside_y = get_wall_sizes(field, interface_normal)

offset = 5e-2
plot_quantities(wall_y_early, wall_y_late, wall_x_early, wall_x_late, offset, scale=1)
plot_quantities(wall_y_early + dside_y, wall_y_late + dside_y, wall_x_early + dside_x, wall_x_late + dside_x, 2*offset, scale=1)

In [ ]:
V_y_early, V_y_late, V_x_early, V_x_late = get_advected_volume(field, interface_normal)
assert np.all(V_x_early[0, :] <= eps)
assert np.all(V_y_early[:, 0] <= eps)
assert np.all(V_x_late[-1, :] <= eps)
assert np.all(V_y_late[:, -1] <= eps)
fig, axes = plt.subplots(2, 2)
axes = axes.flatten()
for ax in axes:
    ax.imshow(field['f'].T, origin="lower", alpha=0.3)
axes[0].quiver(x_grid, y_grid - 0.5, np.zeros_like(V_y_early), -V_y_early)
axes[0].set_title('V_y_early')
axes[1].quiver(x_grid - 0.5, y_grid, -V_x_early, np.zeros_like(V_x_early))
axes[1].set_title('V_x_early')
axes[2].quiver(x_grid, y_grid + 0.5, np.zeros_like(V_y_late), V_y_late)
axes[2].set_title('V_y_late')
axes[3].quiver(x_grid + 0.5, y_grid, V_x_late, np.zeros_like(V_x_late))

In [ ]:
V_u, V_v = get_upwind_advected_fluxes(field, V_y_early, V_y_late, V_x_early, V_x_late)
plt.quiver(x_ugrid, y_ugrid, V_u, np.zeros_like(V_u))
plt.quiver(x_vgrid, y_vgrid, np.zeros_like(V_v), V_v)
plt.imshow(field['f'].T, origin="lower", alpha=0.3)

In [ ]:
np.max(np.abs(V_u)), np.max(np.abs(V_v))

In [ ]:
fig, axes = plt.subplots(2, 3)

C_star = field['f'].copy()
plot_img(C_star, (fig, axes[0][0]))
C_star -= (dt / dx) * deriv(V_u, d_.x, dx)
plot_img(C_star, (fig, axes[0][1]))
C_star[field['f'] >= 0.5] += (dt / dx) * deriv(field['u'], d_.x, dx)[field['f'] >= 0.5]
plot_img(C_star, (fig, axes[0][2]))

C_next = C_star.copy()
plot_img(C_next, (fig, axes[1][0]))
C_next -= (dt / dx) * deriv(V_v, d_.y, dx)
plot_img(C_next, (fig, axes[1][1]))
C_next[field['f'] >= 0.5] += (dt / dx) * deriv(field['v'], d_.y, dx)[field['f'] >= 0.5]
plot_img(C_next, (fig, axes[1][2]))

In [ ]:
plot_field(field)

# Full algorithm

In [ ]:
def navier_stokes_vof_step(field):
    update_velocity(field, dx=dx, dt=dt)
    interface_normal = get_interface_normal(field)
    V_y_early, V_y_late, V_x_early, V_x_late = get_advected_volume(field, interface_normal)
    V_u, V_v = get_upwind_advected_fluxes(field, V_y_early, V_y_late, V_x_early, V_x_late)
    C_next = split_scheme(field, V_u, V_v)
    check(C_next)
    # TODO: this shouldn't be necessary
    C_next = np.clip(C_next, 0, 1)
    field['f'] = C_next

In [ ]:
navier_stokes_vof_step(field)
plot_field(field)

In [ ]:
%matplotlib inline

In [ ]:
from matplotlib.animation import FuncAnimation
from functools import partial
from IPython.display import HTML

fig, axes = plt.subplots(1, 3)
axes = axes.flatten()
fig_, imshow, quiver = plot_field(field, (fig, axes[:2]))
assert fig_ is fig
volume_evo, = axes[2].plot(np.array([np.sum(field['f'])]))

def animate_fig(frame: dict):
    volume_hist = list(volume_evo.get_ydata())
    volume_hist.append(np.sum(frame['f'].T))
    #print(volume_hist)
    volume_evo.set_xdata(range(len(volume_hist)))
    volume_evo.set_ydata(volume_hist)
    axes[2].relim()
    axes[2].autoscale_view()
    imshow.set_array(frame['f'].T)
    u = avg(field['u'], d_.x)
    v = avg(field['v'], d_.y)
    quiver.set_UVC(U=u, V=v)
    return volume_evo, quiver, imshow

def frames(scenario, num_frames: int = 100):
    field = reset_field(scenario)
    for i in range(num_frames):
        navier_stokes_vof_step(field)
        yield field

num_frames = 100
animation = FuncAnimation(fig, animate_fig, frames=frames('circle', num_frames), save_count=num_frames)
HTML(animation.to_jshtml())

In [ ]:
field = reset_field('surface')
np.sum(field['f'])

In [ ]:
np.sum(imshow.get_array())

In [ ]:
volume_evo.get_ydata()